
### Setting storage account info

In [ ]:
contname = 'team4-project2' #azure storage account container
storage_acct_name = '20230821desa'

file_pattern = f"abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/SilverLayer"
import os



### Loading in tables 

- these are the tables for the gold layer

In [ ]:
master_df = spark.read.format('parquet').load(f'{file_pattern}/master_events').withColumnRenamed('id', 'event_id')
actor_df = spark.read.format('parquet').load(f'{file_pattern}/actors')
repo_df = spark.read.format('parquet').load(f'{file_pattern}/repo')
push_df = spark.read.format('parquet').load(f'{file_pattern}/push_event')
commits_df = spark.read.format('parquet').load(f'{file_pattern}/push_commits')
create_event_df = spark.read.format('parquet').load(f'{file_pattern}/create_event')
watch_df = spark.read.format('parquet').load(f'{file_pattern}/watch_events')



### Storage Account Settings

In [ ]:

adls_container = os.getenv('ADLS_CONTAINER_NAME') #azure storage account container
adls_account_name = os.getenv('ADLS_ACCOUNT_NAME')
client_id = os.getenv('ADLS_CLIENT_ID') #aka: app id
tenant_id = os.getenv('ADLS_TENANT_ID') #aka: directory ids

service_credential = dbutils.secrets.get(scope="databricks-app-kv",key="databricks-application")

spark.conf.set(f"fs.azure.account.auth.type.{adls_account_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{adls_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{adls_account_name}.dfs.core.windows.net", f"{client_id}")
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{adls_account_name}.dfs.core.windows.net", service_credential)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{adls_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")



### Writing tables 

- these are the tables written to the gold layer

In [ ]:

master_df.repartition(23).write.mode("overwrite").parquet(f'abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/GoldLayer/master_events')


actor_df.repartition(7).write.mode("overwrite").parquet(f'abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/GoldLayer/actor')


repo_df.repartition(1).write.mode("overwrite").parquet(f'abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/GoldLayer/repo')


push_df.repartition(18).write.mode("overwrite").parquet(f'abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/GoldLayer/push')


commits_df.repartition(7).write.mode("overwrite").parquet(f'abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/GoldLayer/commits')

create_event_df.repartition(3).write.mode("overwrite").parquet(f'abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/GoldLayer/create_event')

watch_df.repartition(3).write.mode("overwrite").parquet(f'abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/GoldLayer/watch_event')


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-173403117479033>:1
----> 1 master_df.repartition(X).write.mode("overwrite").parquet(f'abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/GoldLayer/master_events')
      2 actor_df.repartition(X).write.mode("overwrite").parquet(f'abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/GoldLayer/actor')
      3 repo_df.repartition(X).write.mode("overwrite").parquet(f'abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/GoldLayer/repo')

NameError: name 'master_df' is not defined


#### Specific Use Case:

Analyze the number of watchers a repositiory has to see the new up and coming coding projects.